In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.stats.api as sms

In [87]:
### Production and Prices

dfs = []

# Loop through the years from 2015 to 2023
for i in range(2015, 2024):
    # Construct the file path for each year
    file_path = f'energy-charts_Public_net_electricity_generation_in_Germany_in_{i}.csv'
    
    # Read the CSV file for the current year and append to the list
    df = pd.read_csv(file_path, skiprows=[1]) 
    df['Hour'] = df['Date (GMT+1)'].str.split('T').str[1].str.split('+').str[0].str.split(':').str[0]
    df['Date'] = df['Date (GMT+1)'].str.split('T').str[0]
    # Convert 15-minute MW to hourly MW
    df_hourly = df.groupby(['Date', 'Hour']).sum()
    
    
    
    # Convert columns to numeric if they contain strings
    df_hourly[['Wind offshore','Wind onshore','Solar','Load']]
    # Divide the sum of power generation values by 4
    df_hourly[['Wind offshore','Wind onshore','Solar','Load']] = df_hourly[['Wind offshore','Wind onshore','Solar','Load']] / 4
    df_hourly.reset_index(inplace=True)
    
    
    df_hourly['Date'] = pd.to_datetime(df_hourly['Date'])

    # Convert 'Hour' column to string and pad single-digit hours with '0' to maintain a consistent format
    df_hourly['Hour'] = df_hourly['Hour'].astype(str).str.zfill(2)

    # Merge 'Date' and 'Hour' into a new column 'DateHour' and then drop 'Hour' column
    df_hourly['Date'] = df_hourly['Date'].dt.strftime('%Y-%m-%d') + ' ' + df_hourly['Hour'] + ':00:00'
    df_hourly['Date'] = pd.to_datetime(df_hourly['Date'])  # Convert to datetime format
    df_hourly = df_hourly.drop(columns=['Hour'])
    df_hourly.drop(columns=['Date (GMT+1)'], inplace=True)
    dfs.append(df_hourly)

# Concatenate all dataframes into a single dataframe
Power_production = pd.concat(dfs, ignore_index=True)
Renewable = Power_production[['Date','Wind offshore','Wind onshore','Solar','Load']] 

dfe = []

# Loop through the years from 2015 to 2023
for i in range(2015, 2024):
    # Construct the file path for each year
    file_path = f'energy-charts_Electricity_production_and_spot_prices_in_Germany_in_{i}.csv'
    
    # Read the CSV file for the current year and append to the list
    df = pd.read_csv(file_path, skiprows=[1])
    df['Hour'] = df['Date (GMT+1)'].str.split('T').str[1].str.split('+').str[0].str.split(':').str[0]
    df['Date'] = df['Date (GMT+1)'].str.split('T').str[0]
    df['Date'] = pd.to_datetime(df['Date'])

    # Convert 'Hour' column to string and pad single-digit hours with '0' to maintain a consistent format
    df['Hour'] = df['Hour'].astype(str).str.zfill(2)

    # Merge 'Date' and 'Hour' into a new column 'DateHour' and then drop 'Hour' column
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Hour'] + ':00:00'
    df['Date'] = pd.to_datetime(df['Date'])  # Convert to datetime format
    df = df.drop(columns=['Hour'])
    df.drop(columns=['Date (GMT+1)'], inplace=True)
    dfe.append(df)

# Concatenate all dataframes into a single dataframe
Power_prices = pd.concat(dfe, ignore_index=True)
Power_prices = Power_prices[['Date','Day Ahead Auction']]

In [88]:
Power_DATA = pd.merge(Power_prices, Renewable, on='Date', how='inner')

#### Finding capture rates

In [119]:
import pandas as pd

# Step 1: Extract the date from the datetime column
Power_DATA['Date'] = pd.to_datetime(Power_DATA['Date']).dt.date

# Step 2: Group by date and calculate the total revenue and production for each renewable energy source
grouped = Power_DATA.groupby('Date').agg(
    TotalRevenueWindOffshore=('Wind offshore', lambda x: (x * Power_DATA.loc[x.index, 'Day Ahead Auction']).sum()),
    TotalProductionWindOffshore=('Wind offshore', 'sum'),
    TotalRevenueWindOnshore=('Wind onshore', lambda x: (x * Power_DATA.loc[x.index, 'Day Ahead Auction']).sum()),
    TotalProductionWindOnshore=('Wind onshore', 'sum'),
    TotalRevenueSolar=('Solar', lambda x: (x * Power_DATA.loc[x.index, 'Day Ahead Auction']).sum()),
    TotalProductionSolar=('Solar', 'sum'),
    TotalRevenueBaseload=('Load', lambda x: (x * Power_DATA.loc[x.index, 'Day Ahead Auction']).sum()),
    BaseloadProduction=('Load', 'sum')
)

# Step 3: Calculate the average price (capture price) for each renewable energy source
grouped['Capture_Price_WindOffshore'] = grouped['TotalRevenueWindOffshore'] / grouped['TotalProductionWindOffshore']
grouped['Capture_Price_WindOnshore'] = grouped['TotalRevenueWindOnshore'] / grouped['TotalProductionWindOnshore']
grouped['Capture_Price_Solar'] = grouped['TotalRevenueSolar'] / grouped['TotalProductionSolar']
grouped['Baseload_Price'] = grouped['TotalRevenueBaseload'] / grouped['BaseloadProduction']

# Step 4: Calculate the capture factor compared to baseload production
grouped['Capture_Factor_Wind_Offshore'] = grouped['Capture_Price_WindOffshore'] / grouped['Baseload_Price']
grouped['Capture_Factor_Wind_Onshore'] = grouped['Capture_Price_WindOnshore'] / grouped['Baseload_Price']
grouped['Capture_Factor_Solar'] = grouped['Capture_Price_Solar'] / grouped['Baseload_Price']
grouped.reset_index(inplace=True)

# Step 5: Display the results
Capture_factors = grouped[['Date','Capture_Factor_Wind_Offshore', 'Capture_Factor_Wind_Onshore', 'Capture_Factor_Solar']]
Capture_prices = grouped[['Date','Capture_Price_WindOffshore', 'Capture_Price_WindOnshore', 'Capture_Price_Solar','Baseload_Price']]
Capture_factors['Date'] = pd.to_datetime(Capture_factors['Date'])
Capture_factors
Capture_prices

C:\Users\Bruger\AppData\Local\Temp\ipykernel_5532\3976882828.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Capture_factors['Date'] = pd.to_datetime(Capture_factors['Date'])


,Date,Capture_Price_WindOffshore,Capture_Price_WindOnshore,Capture_Price_Solar,Baseload_Price
0,2015-01-01,35.954173,37.052867,41.924398,36.550475
1,2015-01-02,37.475910,36.067422,42.223400,37.440672
2,2015-01-03,35.694937,34.951322,41.624366,37.255252
3,2015-01-04,35.525522,35.108165,41.915625,36.958398
4,2015-01-05,32.465778,32.474544,41.420387,37.677882
...,...,...,...,...,...
3282,2023-12-27,57.600000,52.507592,74.730591,62.041669
3283,2023-12-28,18.097318,17.055332,16.516223,18.160128
3284,2023-12-29,7.410767,6.142629,1.019475,6.725511
3285,2023-12-30,30.499861,26.656958,26.382954,35.912921


### Volatility making it staionary 

In [117]:
# Convert Date to datetime format
Power_DATA['Date'] = pd.to_datetime(Power_DATA['Date'])

# Extract day, year, and month from the Date column
Power_DATA['Day'] = Power_DATA['Date'].dt.day
Power_DATA['Year'] = Power_DATA['Date'].dt.year
Power_DATA['Month'] = Power_DATA['Date'].dt.month

# Calculate the standard deviation of 'Day Ahead Auction' grouped by Year, Month, and Day
volatility = Power_DATA.groupby(['Year', 'Month', 'Day'])['Day Ahead Auction'].std()

# Weighted standard deviation
weighted_volatility = Power_DATA.groupby(['Year', 'Month', 'Day']).apply(
    lambda x: np.sqrt(np.average((x['Day Ahead Auction'] - x['Day Ahead Auction'].mean()) ** 2, weights=x['Load']))
)

# Make dataframe
volatility_reset = volatility.reset_index()
volatility_reset.columns = ['Year', 'Month', 'Day', 'Volatility']
volatility_reset['Date'] = pd.to_datetime(volatility_reset[['Year', 'Month', 'Day']])
volatility_reset = volatility_reset.drop(columns=['Year', 'Month', 'Day'])
volatility_reset


# Make dataframe
weighted_volatility_reset = weighted_volatility.reset_index()
weighted_volatility_reset.columns = ['Year', 'Month', 'Day', 'Weighted_volatility']
weighted_volatility_reset['Date'] = pd.to_datetime(weighted_volatility_reset[['Year', 'Month', 'Day']])
weighted_volatility_reset = weighted_volatility_reset.drop(columns=['Year', 'Month', 'Day'])
weighted_volatility_reset['Date'] = pd.to_datetime(weighted_volatility_reset['Date'])
weighted_volatility_reset['Weighted_volatility_adj'] = weighted_volatility_reset['Weighted_volatility'].pct_change().dropna()
weighted_volatility_reset.dropna(inplace=True)
weighted_volatility_reset






,Weighted_volatility,Date,Weighted_volatility_adj
1,12.155596,2015-01-02,-0.030222
2,12.292289,2015-01-03,0.011245
3,12.345635,2015-01-04,0.004340
4,12.056345,2015-01-05,-0.023433
5,5.657960,2015-01-06,-0.530707
...,...,...,...
3282,21.386895,2023-12-27,-0.117531
3283,16.563178,2023-12-28,-0.225545
3284,9.029815,2023-12-29,-0.454826
3285,25.611030,2023-12-30,1.836274


### Merge

In [120]:
ANALYSE_DATA = pd.merge(Capture_factors, weighted_volatility_reset, on='Date', how='inner')
ANALYSE_DATA

,Date,Capture_Factor_Wind_Offshore,Capture_Factor_Wind_Onshore,Capture_Factor_Solar,Weighted_volatility,Weighted_volatility_adj
0,2015-01-02,1.000941,0.963322,1.127742,12.155596,-0.030222
1,2015-01-03,0.958118,0.938158,1.117275,12.292289,0.011245
2,2015-01-04,0.961230,0.949937,1.134130,12.345635,0.004340
3,2015-01-05,0.861667,0.861899,1.099329,12.056345,-0.023433
4,2015-01-06,1.002306,1.018463,0.933944,5.657960,-0.530707
...,...,...,...,...,...,...
3281,2023-12-27,0.928408,0.846328,1.204523,21.386895,-0.117531
3282,2023-12-28,0.996541,0.939164,0.909477,16.563178,-0.225545
3283,2023-12-29,1.101889,0.913333,0.151583,9.029815,-0.454826
3284,2023-12-30,0.849273,0.742267,0.734637,25.611030,1.836274


### Empirical analysis

In [122]:
# Define the independent variables including the constant
X = sm.add_constant(ANALYSE_DATA['Capture_Factor_Wind_Offshore'])

# Define the dependent variable
y = ANALYSE_DATA['Weighted_volatility_adj']

# Fit the regression model
model_OLS = sm.OLS(y, X).fit()

# Print the summary of the model
print(model_OLS.summary())

# Get the number of observations and parameters
n = len(y)
k = model_OLS.df_model  # Number of parameters including intercept

# Compute the residual sum of squares
rss = np.sum(model_OLS.resid**2)

# Compute AIC
aic = n * np.log(rss / n) + 2 * k

# Compute BIC
bic = n * np.log(rss / n) + k * np.log(n)

print("AIC:", aic)
print("BIC:", bic)

                               OLS Regression Results                              
Dep. Variable:     Weighted_volatility_adj   R-squared:                       0.005
Model:                                 OLS   Adj. R-squared:                  0.004
Method:                      Least Squares   F-statistic:                     15.56
Date:                     Tue, 20 Feb 2024   Prob (F-statistic):           8.15e-05
Time:                             00:36:39   Log-Likelihood:                -3682.9
No. Observations:                     3286   AIC:                             7370.
Df Residuals:                         3284   BIC:                             7382.
Df Model:                                1                                         
Covariance Type:                 nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------